In [1]:
from architecture import *
import json
import math
from tqdm import tqdm_notebook as tqdm
import torch.nn.functional as F

In [2]:
data_dir = "/tmp2/b05902064/data-giga/"
train_path = data_dir + "train_seq.json"
vocab_path = data_dir + "vocab.json"
lm_path = "trainedELMo"
device = torch.device("cuda:1")
vocab = json.load(open(vocab_path))
VOCAB_SIZE = len(vocab)

In [3]:
elmo = torch.load("/tmp2/b05902064/pretrained_ELMo")
LM = torch.load("/tmp2/b05902064/preload_LM")

In [4]:
m = ContextMatcher(vocab, elmo, LM)
m.to(device)

ContextMatcher(
  (LM): LanguageModel(
    (elmo): Elmo(
      (_elmo_lstm): _ElmoBiLm(
        (_token_embedder): _ElmoCharacterEncoder(
          (char_conv_0): Conv1d(16, 32, kernel_size=(1,), stride=(1,))
          (char_conv_1): Conv1d(16, 32, kernel_size=(2,), stride=(1,))
          (char_conv_2): Conv1d(16, 64, kernel_size=(3,), stride=(1,))
          (char_conv_3): Conv1d(16, 128, kernel_size=(4,), stride=(1,))
          (char_conv_4): Conv1d(16, 256, kernel_size=(5,), stride=(1,))
          (char_conv_5): Conv1d(16, 512, kernel_size=(6,), stride=(1,))
          (char_conv_6): Conv1d(16, 1024, kernel_size=(7,), stride=(1,))
          (_highways): Highway(
            (_layers): ModuleList(
              (0): Linear(in_features=2048, out_features=4096, bias=True)
              (1): Linear(in_features=2048, out_features=4096, bias=True)
            )
          )
          (_projection): Linear(in_features=2048, out_features=512, bias=True)
        )
        (_word_embedding): Emb

In [5]:
print("building candidate mapping")
# (vocab, emb)
vocab_size = len(vocab)
batch_size = 512
embeddings = []
total = int(math.ceil(vocab_size/batch_size))
progress = tqdm(range(total), total=total)
for i in progress:
    fr = i*batch_size
    to = min(fr+batch_size, vocab_size)
    indices = torch.arange(fr, to).unsqueeze(1)
    embeddings.append(m.embed(indices.to(device)) )

# (n_batch, batch_size)
embeddings = torch.cat(embeddings, dim=0)
print(embeddings.shape)
# (vocab, emb)

building candidate mapping



torch.Size([50000, 1, 1024])


In [6]:
embeddings = F.normalize(embeddings, p=2, dim=-1)
embeddings = embeddings.squeeze()
print(embeddings.shape)

torch.Size([50000, 1024])


In [7]:
# embeddings = torch.matmul(embeddings, embeddings.transpose(0, 1))
# print(embeddings.shape)

In [8]:
# embeddings = torch.argsort(embeddings, dim=-1, descending=True)
# embeddings = embeddings[:, :100]

In [13]:
output = []
for i in tqdm(range(50000), total=50000):
    # (50000, 1024)
    cur = embeddings[i:i+1] #(1, 1024)
    scores = torch.matmul(cur, embeddings.transpose(0, 1)) #(1, 50000)
    candidates = torch.argsort(scores, dim=-1, descending=True) #(1, 50000)
    output.append(candidates[0,:50].cpu())

In [14]:
output = torch.stack(output, dim=0)

In [15]:
torch.save(output, "/tmp2/b05902064/candidate_mappings")